Dataset:

Het is een audiodataset. Een audio dataset heeft 3 dimensies. In pytorch is dat de 1d dimensie.

10 klassen:
13 features

8800 observaties
10 getallen
10 herhalingen
88 sprekers (88x10x10)

1 regel getallen is 1 uitspraak van 1 persoon
1 blok is 1 persoon die 1 uitspraak doet (getal 3 bijv)

Potentieel probleem: Te weinig observaties? Te weinig sprekers? Relatief kleine dataset.

Hoe bepaal je de grootte van de filters?
- Relatief kleine filters gebruiken omdat de datasets maar 13 features hebben.
- input channels: Gelijk aan het aantal features. In dit geval 13.




Modellen om te proberen:
CNN
Gru 
Gru + attention

